# MNIST 手寫數字辨識 (MNIST_Autoencoder_TF)

2017/07/23  
徐仕杰

### 目標

** Motivation **   
- 有大量的Labeling Data -> Supervising learning, ex: MNIST Data set    
- 當labeling data很少時怎麼辦？ ex:只有1000張labeled Mnist圖片,  但是有49000張unlabeled的Mnist圖片  
    1. 只用1000張labeled Mnist圖片訓練 -> Supervised Learning, 正確率會很低, (DNN 784->600->480->10, acc: 87.6%)  

    2. 用全部的資料訓練
      - 利用49000張unlabeled的Mnist圖片先找到一些圖片共通的特徵 -> Unsupervised learning  
         - encoder 784->500->250->30  
         - decoder 30->250->500->784
      - 再用1000張labeled Mnist圖片訓練  
         - Encoder + Logistic Regression: 30->10, acc: 93.1％  


** How? **  
- 一般方法: PCA  
- $\color{blue}{NN方法：Autoencoder (Deep Autoencoder)}$


** Network Structure **  
- Autoencoder + 
- Classifier


![](img/autoencoder_1.png)


![](img/autoencoder_2.png)

>Reference: Hinton, Geoffrey E., and Ruslan R. Salakhutdinov. "Reducing the dimensionality of data with neural networks." Science 313.5786 (2006): 504-507   

>李宏毅:會加上比 input 寬的 layer 是為了考慮 feature 不同的 dimension 之間的 interaction ，就好像是 SVM 的 kernel 一樣，先把輸入投影到高維空間

### Tips:
- 記得要download data set: 
[Mnist](https://github.com/Backlu/tf-keras-tutorial/blob/master/basic/mnist.pkl.xz)
- 在command前面加** ! **可以執行console command
- 在command前面加** ? **可以查詢Help
- 什麼是one-hot representation:
[one-hot](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science)  
- 好用的markdown語法
[markdown](https://www.zybuluo.com/codeep/note/163962#1如何输入一个方程式序列)  
<br>
- import PIL error : pip install Pillow
- import pandas error: pip install pandas
- import lzma error: 請用python 3



## Outline

-  [Import Package & Functions](#import) 
-  [1. Import MNIST Data](#Data) 
-  [2. Modeling](#Model)  
-  [3. Training](#Training)
-  [4. Visualization- Tensorboard](#visualization)

<a id='import'></a>
## Import Package & Functions

In [1]:
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tfdot import tfdot
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import matplotlib.pyplot as plt
from PIL import Image
import warnings
from sklearn import preprocessing
from functools import reduce

In [2]:
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))

In [3]:
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()

In [4]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  

In [5]:
def weight_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)
def bias_variable(shape, name):
    return tf.Variable(tf.truncated_normal(shape = shape, stddev = 0.1), name)

In [6]:
def kl_div(rho, rho_hat):
    invrho = tf.subtract(tf.constant(1.), rho)
    invrhohat = tf.subtract(tf.constant(1.), rho_hat)
    logrho = tf.add(logfunc(rho,rho_hat), logfunc(invrho, invrhohat))
    return logrho
    
def logfunc(x, x2):
    return tf.multiply( x, tf.log(tf.div(x,x2)))

<a id='Data'></a>
## 1. Import MNIST Data

#### 先把MNIST資料讀進來
- Training Data: 訓練Model
- Validataion Data: 訓練Model的時候, 同步監控目前模型的好壞
- Testing Data: 訓練結束後, 評估模型的好壞

In [7]:
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

print('list裡的前面是picture X',train_set[0])
print('後面是label Y',train_set[1])


list裡的前面是picture X [[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
後面是label Y [5 0 4 ..., 8 4 8]


In [8]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
print('training data size:',len(train_X))
print('validataion data size:',len(validation_X))
print('testing data size:',len(test_X))
print('picture shape:',train_X[0].shape)

training data size: 50000
validataion data size: 10000
testing data size: 10000
picture shape: (784,)


#### 把Y label變成one-hot representation

In [9]:
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

<a id='Model'></a>
## 2. Modeling


- Autoencoder 
    -  [Parameters](#定義參數) 
    -  [Autoencoder modeling - Network Structure](#設計一個) 
    -  [Autoencoder modeling - Loss Function & Optimizer](#選一個loss) 
    -  [Autoencoder training](#選一個o) 
    -  [Autoencoder - test](#開始執行)   
- Classifier       
    -  [Classifier modeling - Network Structure](#算一下正)  
    -  [Classifier modeling - loss function and Optimizer](#)
    -  [Classifier training](#)
    -  [Classifier testing](#)

** Parameters **

In [10]:
tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, 784], name="X")
Y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Y_")

** Autoencoder modeling - Network Structure **

encoded_neural = 30
def build_sae():
    W_e_1 = weight_variable([784, 1000], "w_e_1")
    b_e_1 = bias_variable([1000], "b_e_1")
    h_e_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W_e_1), b_e_1))

    W_e_2 = weight_variable([1000, 500], "w_e_2")
    b_e_2 = bias_variable([500], "b_e_2")
    h_e_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_1, W_e_2), b_e_2))
    
    W_e_3 = weight_variable([500, 250], "w_e_3")
    b_e_3 = bias_variable([250], "b_e_3")
    h_e_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_2, W_e_3), b_e_3))
    
    W_e_4 = weight_variable([250, encoded_neural], "w_e_4")
    b_e_4 = bias_variable([encoded_neural], "b_e_4")
    h_e_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_3, W_e_4), b_e_4))

    W_d_1 = weight_variable([encoded_neural, 250], "w_d_1")
    b_d_1 = bias_variable([250], "b_d_1")
    h_d_1 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_4, W_d_1), b_d_1))
    
    W_d_2 = weight_variable([250, 500], 'w_d_2')
    b_d_2 = bias_variable([500], "b_d_2")
    h_d_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_1, W_d_2), b_d_2))

    W_d_3 = weight_variable([500, 1000], "w_d_3")
    b_d_3 = bias_variable([1000], "b_d_3")
    h_d_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_2, W_d_3), b_d_3))
    
    W_d_4 = weight_variable([1000, 784], "w_d_4")
    b_d_4 = bias_variable([784], "b_d_4")
    h_d_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_3, W_d_4), b_d_4))
    h = [h_e_1, h_e_2, h_e_3, h_e_4, h_d_1, h_d_2, h_d_3, h_d_4]
    w = [W_e_1, W_e_2,W_e_3, W_e_4, W_d_1, W_d_2, W_d_3,  W_d_4]
    
    return h, w, h_e_4, h_d_4

h, w, encoded, x_reconstruct = build_sae()

In [11]:
encoded_neural = 30
def build_sae():
    W_e_1 = weight_variable([784, 500], "w_e_1")
    b_e_1 = bias_variable([500], "b_e_1")
    h_e_1 = tf.nn.sigmoid(tf.add(tf.matmul(X, W_e_1), b_e_1))

    W_e_2 = weight_variable([500, 250], "w_e_2")
    b_e_2 = bias_variable([250], "b_e_2")
    h_e_2 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_1, W_e_2), b_e_2))

    W_e_4 = weight_variable([250, encoded_neural], "w_e_4")
    b_e_4 = bias_variable([encoded_neural], "b_e_4")
    h_e_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_2, W_e_4), b_e_4))

    W_d_1 = weight_variable([encoded_neural, 250], "w_d_1")
    b_d_1 = bias_variable([250], "b_d_1")
    h_d_1 = tf.nn.sigmoid(tf.add(tf.matmul(h_e_4, W_d_1), b_d_1))

    W_d_3 = weight_variable([250, 500], "w_d_3")
    b_d_3 = bias_variable([500], "b_d_3")
    h_d_3 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_1, W_d_3), b_d_3))
    
    W_d_4 = weight_variable([500, 784], "w_d_4")
    b_d_4 = bias_variable([784], "b_d_4")
    h_d_4 = tf.nn.sigmoid(tf.add(tf.matmul(h_d_3, W_d_4), b_d_4))
    h = [h_e_1, h_e_2, h_e_4, h_d_1, h_d_3, h_d_4]
    w = [W_e_1, W_e_2, W_e_4, W_d_1, W_d_3,  W_d_4]
    
    return h, w, h_e_4, h_d_4

h, w, encoded, x_reconstruct = build_sae()

** Autoencoder modeling - Loss Function & Optimizer **

In [12]:
alpha = 5e-6
beta = 7.5e-5
kl_div_loss = reduce(lambda x, y: x + y, map(lambda x: tf.reduce_sum(kl_div(0.02, tf.reduce_mean(x,0))), h))
l2_loss = reduce(lambda x, y: x + y, map(lambda x: tf.nn.l2_loss(x), w))
loss_auto = tf.reduce_mean(tf.pow(x_reconstruct - X, 2)) #+ alpha * l2_loss #+ beta * kl_div_loss

optimizer = tf.train.AdamOptimizer(0.01).minimize(loss_auto)


** Classifier modeling - Network Structure **

In [13]:
def build_classfier(encoded):
    W_cf_1 = weight_variable([30, 10], "W_cf_1")
    b_cf_1 = bias_variable([10], "b_cf_1")
    h_cf_1 = tf.add(tf.matmul(encoded, W_cf_1), b_cf_1)
    h_cf_1_sf = tf.nn.softmax(h_cf_1)
    return [h_cf_1_sf], [W_cf_1,b_cf_1], h_cf_1, h_cf_1_sf

_, trainable_vars, Y, Y_softmax = build_classfier(encoded)
#correct_prediction = tf.equal(Y_softmax_argmax, tf.argmax(Y_, 1))
correct_pred = tf.equal(tf.argmax(Y_softmax, 1), tf.argmax(Y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, "float"))

** Classifier modeling - loss function and Optimizer **

In [14]:
loss_cla = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_, logits=Y))
#optimizer_cla = tf.train.RMSPropOptimizer(0.01).minimize(loss_cla,var_list=trainable_vars)
optimizer_cla = tf.train.RMSPropOptimizer(0.01).minimize(loss_cla)

<a id='Training'></a>
## 3. Training

** Autoencoder training **

In [15]:
init_op = tf.global_variables_initializer()
sess = tf.InteractiveSession()
sess.run(init_op)


In [16]:
train_X_unlabeled = train_X[:49000]
epoch = 30
batch_size = 500
total_batch= len(train_X_unlabeled) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X_unlabeled.shape[0], batch_size, replace=False)
        batch_x = train_X_unlabeled[rnd_idx]
        _, loss_v= sess.run([optimizer, loss_auto], feed_dict={X: batch_x})
    updateProgress('epoch:{x0}, batch:{x4} loss:{x3}'.format(x0=ep,x3=loss_v,x4=i))
    print()


epoch:0, batch:98 loss:0.06745240092277527
epoch:1, batch:98 loss:0.06694648414850235
epoch:2, batch:98 loss:0.06689544022083282
epoch:3, batch:98 loss:0.05971969664096832
epoch:4, batch:98 loss:0.05664415284991264
epoch:5, batch:98 loss:0.04818274825811386
epoch:6, batch:98 loss:0.04321978986263275
epoch:7, batch:98 loss:0.03567231446504593
epoch:8, batch:98 loss:0.030726641416549683
epoch:9, batch:98 loss:0.027781905606389046
epoch:10, batch:98 loss:0.024744395166635513
epoch:11, batch:98 loss:0.024184077978134155
epoch:12, batch:98 loss:0.023515019565820694
epoch:13, batch:98 loss:0.02161537855863571
epoch:14, batch:98 loss:0.021047264337539673
epoch:15, batch:98 loss:0.01966499350965023
epoch:16, batch:98 loss:0.01898932456970215
epoch:17, batch:98 loss:0.018809165805578232
epoch:18, batch:98 loss:0.01764651946723461
epoch:19, batch:98 loss:0.0172840878367424
epoch:20, batch:98 loss:0.01622960716485977
epoch:21, batch:98 loss:0.015551058575510979
epoch:22, batch:98 loss:0.015429872

** Autoencoder - test (mnist) **

In [17]:
showX(test_X[:20])
batch_tx = test_X[:20]
decoded_imgs = sess.run([x_reconstruct], feed_dict={X: batch_tx})

decoded_imgs_1 = np.array(decoded_imgs)
decoded_imgs_1 = decoded_imgs_1.reshape(20,784)
print('')
showX(decoded_imgs_1[:20])

>** Training爛掉時會像這樣：**
![](img/fail.png)

** Autoencoder - test other picture **

In [18]:
import matplotlib.image as mpimg
warnings.filterwarnings('ignore')
piclist = ['img/ae1.jpg','img/ae2.jpg','img/ae3.jpg','img/ae4.jpeg','img/ae5.jpeg']
batch_tx=[]
min_max_scaler = preprocessing.MinMaxScaler()
normalize_scaler = preprocessing.StandardScaler()
for pic in piclist:
    im = Image.open(pic).convert('L') #Can be many different formats.
    #display(im)
    im = im.resize((28,28))
    display(im)
    #img = mpimg.imread(pic)
    #plt.imshow(img)
    #plt.show()
    #pix = im.load()
    pixels = list(im.getdata())
    pixels = np.array(pixels).reshape(784)
    pixels_scaled = normalize_scaler.fit_transform(pixels.reshape((-1, 1)))
    batch_tx.append(pixels_scaled.reshape((784)))
    
decoded_img_test = sess.run([x_reconstruct], feed_dict={X: batch_tx})
decoded_img_test = np.array(decoded_img_test)
decoded_img_test = decoded_img_test.reshape(-1,784)
showX(decoded_img_test[:20])

** Classifier training **

In [26]:
train_X_labeled = train_X[49000:]
train_Y_labeled = train_Y[49000:]
epoch = 30
batch_size = 60
total_batch= len(train_X_labeled) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X_labeled.shape[0], batch_size, replace=False)
        batch_x = train_X_labeled[rnd_idx]
        batch_y = train_Y_labeled[rnd_idx]
        _, loss_v, acc_v= sess.run([optimizer_cla, loss_cla, accuracy], feed_dict={X: batch_x, Y_:batch_y})
    loss_v2, acc_v2= sess.run([loss_cla, accuracy], feed_dict={X: validation_X , Y_: validation_Y})
    updateProgress('Ep:{x0}, Bch:{x4}, Tr: loss:{x5}, acc:{x6}, Vd: loss:{x3}, acc:{x2}'.format(x0=ep,x1=i,x2=acc_v2,x3=loss_v2,x4=i,x6=acc_v,x5=loss_v))
    print()



Ep:0, Bch:16, Tr: loss:0.007412776350975037, acc:1.0, Vd: loss:0.22872737050056458, acc:0.9348000288009644
Ep:1, Bch:16, Tr: loss:0.005457836203277111, acc:1.0, Vd: loss:0.23560687899589539, acc:0.9344000220298767
Ep:2, Bch:16, Tr: loss:0.003175800433382392, acc:1.0, Vd: loss:0.24398218095302582, acc:0.9337000250816345
Ep:3, Bch:16, Tr: loss:0.0018224477535113692, acc:1.0, Vd: loss:0.2595207095146179, acc:0.9340999722480774
Ep:4, Bch:16, Tr: loss:0.0010774016845971346, acc:1.0, Vd: loss:0.27319666743278503, acc:0.9332000017166138
Ep:5, Bch:16, Tr: loss:0.0005443973350338638, acc:1.0, Vd: loss:0.28039342164993286, acc:0.9358000159263611
Ep:6, Bch:16, Tr: loss:0.0003572526329662651, acc:1.0, Vd: loss:0.29637303948402405, acc:0.9362999796867371
Ep:7, Bch:16, Tr: loss:0.0001569773885421455, acc:1.0, Vd: loss:0.3073108494281769, acc:0.9363999962806702
Ep:8, Bch:16, Tr: loss:8.588202035753056e-05, acc:1.0, Vd: loss:0.32244858145713806, acc:0.9355000257492065
Ep:9, Bch:16, Tr: loss:7.25597783

** Classifier testing **

In [27]:
# 抓出所有預測錯誤的
pred_v = Y_softmax.eval(feed_dict={X: test_X})
pred_v = pd.DataFrame(pred_v).apply(np.argmax,axis=1)
pred_r = correct_pred.eval({X: test_X , Y_: test_Y})
acc = sess.run(accuracy, feed_dict={X: test_X , Y_: test_Y})

pred_v_fail = pred_v[pred_r==False]
test_y_fail = test_y[pred_r==False]
test_X_fail = test_X[pred_r==False]

errorqty = len(pred_v_fail)
correctqty = len(pred_v) - errorqty
print('Accuracy:',acc, ', ','{f}/{t}'.format(t=correctqty,f=errorqty))

show_rownum = int(pred_v_fail.shape[0]/10)
for i in range(show_rownum+1):
    start=i
    end = i+10
    print('\npredict: ',pred_v_fail.values[start:end])
    print('answer: ',test_y_fail[start:end])
    showX(test_X_fail[start:end])


Accuracy: 0.9317 ,  683/9317

predict:  [8 6 2 7 4 2 5 7 2 9]
answer:  [3 4 3 9 9 3 6 8 4 4]



predict:  [6 2 7 4 2 5 7 2 9 0]
answer:  [4 3 9 9 3 6 8 4 4 6]



predict:  [2 7 4 2 5 7 2 9 0 9]
answer:  [3 9 9 3 6 8 4 4 6 8]



predict:  [7 4 2 5 7 2 9 0 9 5]
answer:  [9 9 3 6 8 4 4 6 8 8]



predict:  [4 2 5 7 2 9 0 9 5 6]
answer:  [9 3 6 8 4 4 6 8 8 4]


...


predict:  [8 0 7 9 7 7 5 0 9 2]
answer:  [4 8 8 4 9 2 3 2 8 7]



predict:  [0 7 9 7 7 5 0 9 2 9]
answer:  [8 8 4 9 2 3 2 8 7 4]



predict:  [7 9 7 7 5 0 9 2 9 0]
answer:  [8 4 9 2 3 2 8 7 4 6]



predict:  [9 7 7 5 0 9 2 9 0 3]
answer:  [4 9 2 3 2 8 7 4 6 2]



predict:  [7 7 5 0 9 2 9 0 3 8]
answer:  [9 2 3 2 8 7 4 6 2 5]


<a id='visualization'></a>
### Visualization- Tensor Borad

** 先把label存到metadata.tsv **

In [21]:
LOG_DIR = 'tb-log0'
input_data = train_X[:5000]
input_y = train_y[:5000]

if not os.path.exists(LOG_DIR):
    os.mkdir(LOG_DIR)
#把圖片的label存到metadata.tsv
metadata_file_path = os.path.join(LOG_DIR, 'metadata.tsv')
with open(metadata_file_path, 'w') as metadata_file:
    for row in range(len(input_y)):
        c=input_y[row]
        metadata_file.write('{}\n'.format(c))

** 把要在tensorboard上呈現的variable embedding存起來 **

In [28]:
#Save variable
Input_Var = tf.Variable(input_data, name='Input_var')
encoded_Var = tf.Variable(encoded.eval({X: input_data}), name='encoded_var')
x_reconstruct_Var = tf.Variable(x_reconstruct.eval({X: input_data}), name='x_reconstruct_Var')

saver = tf.train.Saver([Input_Var,encoded_Var,x_reconstruct_Var])
sess.run(Input_Var.initializer)
sess.run(encoded_Var.initializer)
sess.run(x_reconstruct_Var.initializer)
saver.save(sess, os.path.join(LOG_DIR, 'autoencoder.ckpt'))

#在config裡面用一個embedding關聯 tensor & its metadata
config = projector.ProjectorConfig()
# One can add multiple embeddings.

embedding1 = config.embeddings.add()
embedding1.tensor_name = Input_Var.name
embedding1.metadata_path = metadata_file_path

embedding2 = config.embeddings.add()
embedding2.tensor_name = encoded_Var.name
embedding2.metadata_path = metadata_file_path

embedding3 = config.embeddings.add()
embedding3.tensor_name = x_reconstruct_Var.name
embedding3.metadata_path = metadata_file_path

#embedding4 = config.embeddings.add()
#embedding4.tensor_name = Out_Var.name
#embedding4.metadata_path = metadata_file_path

#embedding.sprite.image_path = os.path.join(LOG_DIR, 'img/mnist_10k_sprite.png')
# Specify the width and height of a single thumbnail.
#embedding.sprite.single_image_dim.extend([28, 28])



# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)


In [23]:
#!tensorboard --logdir=tb-log0 --port=6006

** epoch 15 **
![epoch15](img/ae_emd_op15.png)

** epoch 30 **
![epoch15](img/ae_emd_op30.png)

** epoch 60 **
![epoch15](img/ae_emd_op60.png)

** epoch 150 **
![epoch15](img/ae_emd_op150.png)

### 進階應用

![](img/autoencoder_3.png)
![](img/autoencoder_4.png)

### Reference
[ML Lecture 13: Unsupervised Learning - Linear Methods](https://www.youtube.com/watch?v=iwh5o_M4BNU)  
[ML Lecture 16: Unsupervised Learning - Auto-encoder](https://www.youtube.com/watch?v=Tk5B4seA-AU)  
[Tensorflow Day16 Autoencoder 實作](http://ithelp.ithome.com.tw/articles/10188177)  
[Hinton's paper Reducing the Dimensionality of Data with Neural Networks](https://www.cs.toronto.edu/~hinton/science.pdf)